In [45]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [31]:
trans_data = pd.read_csv('transaction_data.csv')
cust_data = pd.read_csv('cust_data.csv')
#trans_data

# Preprocessing

In [81]:
# Convert 'Order Date' to datetime format
trans_data['Order Date'] = pd.to_datetime(trans_data['Order Date'])

# Define the treatment date
treatment_date = pd.Timestamp('2019-03-02')

pre_treatment_period = (trans_data['Order Date'] >= treatment_date - pd.Timedelta(days=60)) & \
                       (trans_data['Order Date'] < treatment_date)
post_treatment_period = (trans_data['Order Date'] >= treatment_date) & \
                        (trans_data['Order Date'] < treatment_date + pd.Timedelta(days=60))


# Aggregate profits by customer ID for pre and post-treatment periods
pre_treatment_profit = trans_data[pre_treatment_period].groupby('Customer ID')['Profit'].sum().reset_index(name='Pre-Treatment Profit')
post_treatment_profit = trans_data[post_treatment_period].groupby('Customer ID')['Profit'].sum().reset_index(name='Post-Treatment Profit')


# Merge the aggregated profits with the customer data
merged_data = pd.merge(cust_data, pre_treatment_profit, on='Customer ID', how='left')
merged_data = pd.merge(merged_data, post_treatment_profit, on='Customer ID', how='left')

# Fill NaN values with 0 for customers with no transactions in a given period
merged_data.fillna(0, inplace=True)

# Display the first few rows of the merged data
print(merged_data)

       Customer ID State Treatment Group  Pre-Treatment Profit  \
0                0    VA         No Mail                  0.00   
1                1    VA            Mail                 27.94   
2                2    VA            Mail                530.79   
3                3    OH            Mail                  0.00   
4                4    OH            Mail                 54.72   
...            ...   ...             ...                   ...   
11995        11995    GA         No Mail                  0.00   
11996        11996    GA         No Mail                146.98   
11997        11997    OH            Mail                  0.00   
11998        11998    OH            Mail                  6.92   
11999        11999    GA         No Mail                165.04   

       Post-Treatment Profit  
0                       0.00  
1                       0.00  
2                     126.26  
3                       0.00  
4                      99.91  
...                  

In [66]:
pre_treatment_mean = merged_data['Pre-Treatment Profit'].mean()
pre_treatment_std = merged_data['Pre-Treatment Profit'].std()

post_treatment_mean = merged_data['Post-Treatment Profit'].mean()
post_treatment_std = merged_data['Post-Treatment Profit'].std()

print(f"Pre-Treatment Profit: Mean = {pre_treatment_mean:.2f}, Standard Deviation = {pre_treatment_std:.2f}")
print(f"Post-Treatment Profit: Mean = {post_treatment_mean:.2f}, Standard Deviation = {post_treatment_std:.2f}")


Pre-Treatment Profit: Mean = 77.33, Standard Deviation = 229.01
Post-Treatment Profit: Mean = 119.35, Standard Deviation = 337.12


# Before and After

Treatment effect

In [26]:
# Filter for customers in the treatment group
treatment_group = merged_data[merged_data['Treatment Group'] == 'Mail']

# Calculate the mean profit before and after the treatment for the treatment group
mean_pre_treatment_profit = treatment_group['Pre-Treatment Profit'].mean()
mean_post_treatment_profit = treatment_group['Post-Treatment Profit'].mean()

# Estimate the effect of the campaign for the treatment group
treatment_effect = mean_post_treatment_profit - mean_pre_treatment_profit
print('Estimated Treatment Effect is:', treatment_effect)


Estimated Treatment Effect is: 53.17291166666678


Expected gain of marketing campaign

In [69]:
estimated_treatment_effect = 53.17291166666678
remaining_population = 300000
cost_per_catalog = 20

total_incremental_profit = estimated_treatment_effect * remaining_population

total_cost = cost_per_catalog * remaining_population

expected_gain = total_incremental_profit - total_cost

print(f"Expected Gain is: ${expected_gain:.2f}")


Expected Gain is: $9951873.50


# Randomization check

In [32]:

# Perform a t-test on pre-treatment profits between the treatment and control groups
control_group = merged_data[merged_data['Treatment Group'] == 'No Mail']
t_stat, p_value = ttest_ind(treatment_group['Pre-Treatment Profit'], control_group['Pre-Treatment Profit'])

print('The T-test p-value for pre-treatment profits:', p_value)

# Chi-square test for the 'State' distribution between the two groups
contingency_table = pd.crosstab(merged_data['State'], merged_data['Treatment Group'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print('Chi-square test p-value for state distribution:', p_value)


The T-test p-value for pre-treatment profits: 0.2748271847214662
Chi-square test p-value for state distribution: 0.8271959154190364


# Average Treatment Effect (ATE)

In [74]:
# Encode the 'Treatment Group' as a binary variable
merged_data['Treatment'] = merged_data['Treatment Group'].map({'Mail': 1, 'No Mail': 0})

# Prepare the independent variables (add a constant to the model for the intercept)
X = sm.add_constant(merged_data['Treatment'])  
Y = merged_data['Post-Treatment Profit']

# Fit the OLS model
model = sm.OLS(Y, X).fit()

# Extract the ATE (treatment effect), its standard error, and p-value
ate_estimate = model.params['Treatment']
ate_std_error = model.bse['Treatment']
ate_p_value = model.pvalues['Treatment']

print(f"Average Treatment Effect ATE Estimate: {ate_estimate}")
print(f"Standard Error: {ate_std_error}")
print(f"P-value: {ate_p_value}")



Average Treatment Effect ATE Estimate: 17.74328499999995
Standard Error: 6.153044015728501
P-value: 0.003937822111943358


In [79]:
ate = 17.74328499999995
cost_per_catalog = 20
remaining_population = 300000

total_additional_profit = ate * remaining_population

total_cost = cost_per_catalog * remaining_population

expected_gain = total_additional_profit - total_cost
print(f"The expected gain of the campaign is: ${expected_gain}")

The expected gain of the campaign is: $-677014.5000000149


# Difference-in-Difference (Diff-in-Diff) Estimator

In [51]:
import statsmodels.formula.api as smf

# Ensure the DataFrame is correctly defined for this step
diff_in_diff_data['Profit_Change'] = diff_in_diff_data['Post-Treatment Profit'] - diff_in_diff_data['Pre-Treatment Profit']
diff_in_diff_data['Treatment'] = diff_in_diff_data['Treatment Group'].apply(lambda x: 1 if x == 'Mail' else 0)

# Perform the Diff-in-Diff analysis using OLS regression
diff_in_diff_model = smf.ols('Profit_Change ~ Treatment', data=diff_in_diff_data).fit()

# Display the regression results
print(diff_in_diff_model.summary())



                            OLS Regression Results                            
Dep. Variable:          Profit_Change   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     27.81
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           1.36e-07
Time:                        19:52:59   Log-Likelihood:                -82372.
No. Observations:               12000   AIC:                         1.647e+05
Df Residuals:                   11998   BIC:                         1.648e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.8637      2.991     10.318      0.0

In [80]:
treatment_effect = 22.3092
remaining_population = 300000
cost_per_catalog = 20

total_additional_profit = treatment_effect * remaining_population
total_cost = cost_per_catalog * remaining_population
expected_gain = total_additional_profit - total_cost
print(f"The expected gain of the campaign is: ${expected_gain}")

The expected gain of the campaign is: $692760.0


# Basic Targeting

a. Compute ATE

In [52]:
states = merged_data['State'].unique()
ate_by_state = {}

for state in states:
    state_data = merged_data[merged_data['State'] == state]
    treatment_group_state = state_data[state_data['Treatment Group'] == 'Mail']
    control_group_state = state_data[state_data['Treatment Group'] == 'No Mail']
    
    ate_state = treatment_group_state['Post-Treatment Profit'].mean() - control_group_state['Post-Treatment Profit'].mean()
    ate_by_state[state] = ate_state

print('ATE by State:', ate_by_state)


ATE by State: {'VA': 0.9595255780464811, 'OH': 18.169995979882685, 'GA': 31.161070753286793}


c. Compute Expected gain of the marketing campaign

In [64]:
cust_data1 = pd.read_csv('cust_data.csv')

# Calculate the frequency of each state in the sample
state_distribution = cust_data1['State'].value_counts(normalize=True)

remaining_population = 300000  # Remaining customer base outside of the sample

estimated_state_distribution = state_distribution * remaining_population

# ATEs by state from your previous results
ate_by_state = {'VA': 0.9595255780464811, 'OH': 18.169995979882685, 'GA': 31.161070753286793}

cost_per_catalog = 20

# Calculate the expected gain for each state where the campaign is profitable
expected_gain = 0
for state, proportion in estimated_state_distribution.items():
    ate = ate_by_state.get(state, 0)  # Get ATE for the state, defaulting to 0 if not found
    if ate > cost_per_catalog:  # Only consider states where ATE exceeds the cost
        gain_per_customer = ate - cost_per_catalog
        expected_gain += gain_per_customer * proportion

print(f"Expected Gain is: ${expected_gain:.2f}")


Expected Gain is: $1206790.78
